In [69]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [70]:
data = pd.read_csv('../data/tweets_with_labels.csv')
tweets, label = data["tweets"], data["labels"]

In [71]:
dataset = tf.data.Dataset.from_tensor_slices((tweets, label))

In [72]:
max_tokens = tweets.str.split().str.len().max() + 1

In [73]:
tweets_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

tweets_vectorizer.adapt(tweets)

model = tf.keras.Sequential([
    tweets_vectorizer,
    tf.keras.layers.Embedding(max_tokens, 64, mask_zero=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [74]:
shuffle_dataset = dataset.shuffle(10_000)

train_dataset = shuffle_dataset.take(7_200)
validation_dataset = shuffle_dataset.skip(7200).take(800)
test_dataset = shuffle_dataset.skip(8000).take(2000)

In [75]:
cached_train = train_dataset.shuffle(1024).batch(1024).cache()
cached_validation = validation_dataset.batch(1024).cache()
cached_test = test_dataset.batch(512).cache()

In [76]:
history = model.fit(cached_train, epochs=30, validation_data=cached_validation)

Epoch 1/30
8/8 [==============================] - 1s 39ms/step - loss: 0.6729 - accuracy: 0.7756 - val_loss: 0.6542 - val_accuracy: 0.7513
Epoch 2/30
8/8 [==============================] - 0s 24ms/step - loss: 0.6354 - accuracy: 0.7760 - val_loss: 0.6189 - val_accuracy: 0.7513
Epoch 3/30
8/8 [==============================] - 0s 23ms/step - loss: 0.5958 - accuracy: 0.7760 - val_loss: 0.5841 - val_accuracy: 0.7513
Epoch 4/30
8/8 [==============================] - 0s 21ms/step - loss: 0.5580 - accuracy: 0.7760 - val_loss: 0.5548 - val_accuracy: 0.7513
Epoch 5/30
8/8 [==============================] - 0s 22ms/step - loss: 0.5271 - accuracy: 0.7760 - val_loss: 0.5348 - val_accuracy: 0.7513
Epoch 6/30
8/8 [==============================] - 0s 23ms/step - loss: 0.5064 - accuracy: 0.7760 - val_loss: 0.5238 - val_accuracy: 0.7525
Epoch 7/30
8/8 [==============================] - 0s 23ms/step - loss: 0.4943 - accuracy: 0.7761 - val_loss: 0.5164 - val_accuracy: 0.7525
Epoch 8/30
8/8 [===========

In [77]:
model.evaluate(cached_test)

4/4 [==============================] - 0s 7ms/step - loss: 0.2226 - accuracy: 0.9080


[0.22256051003932953, 0.9079999923706055]

In [85]:
model.predict(["I'm depression"])

array([[0.9927988]], dtype=float32)

In [87]:
label.value_counts()
# there 2314 depressed tweets
# the word 'depression' show up 2579 times

0    8000
1    2314
Name: labels, dtype: int64